In [0]:
!pip install -U keras-tuner
import numpy as np
import pandas as pd
import sklearn
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import roc_auc_score, auc, precision_recall_curve, accuracy_score
from joblib import dump, load

from tensorflow import keras
from tensorflow.keras import optimizers, losses, activations, models
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, LearningRateScheduler, ReduceLROnPlateau
from tensorflow.keras.layers import Dense, Input, Dropout, Convolution1D, MaxPool1D, GlobalMaxPool1D, GlobalAveragePooling1D, \
    concatenate,SimpleRNN,LSTM,Embedding,GRU,Bidirectional,Masking

from kerastuner.tuners import Hyperband
from kerastuner.engine.hypermodel import HyperModel
from kerastuner.engine.hyperparameters import HyperParameters

from google.colab import drive
drive.mount('/content/drive/')

#https://keras-team.github.io/keras-tuner/examples/helloworld/

     |████████████████████████████████| 61kB 9.6MB/s 
  Created wheel for keras-tuner: filename=keras_tuner-1.0.1-cp36-none-any.whl size=73200 sha256=a10e5e824b118d42ff6f6ec2f985bee67c24dd14e3661f14c9c039924ebf3f1a
  Stored in directory: /root/.cache/pip/wheels/b9/cc/62/52716b70dd90f3db12519233c3a93a5360bc672da1a10ded43
  Created wheel for terminaltables: filename=terminaltables-3.1.0-cp36-none-any.whl size=15356 sha256=0885e5eed8f3a08979cbad9af775986211810c0bfd6bfa18a2a2df9c6c0b35cf
  Stored in directory: /root/.cache/pip/wheels/30/6b/50/6c75775b681fb36cdfac7f19799888ef9d8813aff9e379663e
Successfully built keras-tuner terminaltables


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


In [0]:
import os
os.chdir("/content/drive/My Drive/Healthcare")

In [0]:
df_1 = pd.read_csv("ptbdb_normal.csv", header=None)

df_2 = pd.read_csv("ptbdb_abnormal.csv", header=None)

df = pd.concat([df_1, df_2])

df_train, df_test = train_test_split(df, test_size=0.2, random_state=1337, stratify=df[187])

Y = np.array(df_train[187].values).astype(np.int8)
X = np.array(df_train[list(range(187))].values)[..., np.newaxis]

Y_test = np.array(df_test[187].values).astype(np.int8)
X_test = np.array(df_test[list(range(187))].values)[..., np.newaxis]


def get_model():
    nclass = 1
    inp = Input(shape=(187, 1))
    img_1 = Convolution1D(16, kernel_size=5, activation=activations.relu, padding="valid")(inp)
    img_1 = Convolution1D(16, kernel_size=5, activation=activations.relu, padding="valid")(img_1)
    img_1 = MaxPool1D(pool_size=2)(img_1)
    img_1 = Dropout(rate=0.1)(img_1)
    img_1 = Convolution1D(32, kernel_size=3, activation=activations.relu, padding="valid")(img_1)
    img_1 = Convolution1D(32, kernel_size=3, activation=activations.relu, padding="valid")(img_1)
    img_1 = MaxPool1D(pool_size=2)(img_1)
    img_1 = Dropout(rate=0.1)(img_1)
    img_1 = Convolution1D(32, kernel_size=3, activation=activations.relu, padding="valid")(img_1)
    img_1 = Convolution1D(32, kernel_size=3, activation=activations.relu, padding="valid")(img_1)
    img_1 = MaxPool1D(pool_size=2)(img_1)
    img_1 = Dropout(rate=0.1)(img_1)
    img_1 = Convolution1D(256, kernel_size=3, activation=activations.relu, padding="valid")(img_1)
    img_1 = Convolution1D(256, kernel_size=3, activation=activations.relu, padding="valid")(img_1)
    img_1 = GlobalMaxPool1D()(img_1)
    img_1 = Dropout(rate=0.2)(img_1)

    dense_1 = Dense(64, activation=activations.relu, name="dense_1")(img_1)
    dense_1 = Dense(64, activation=activations.relu, name="dense_2")(dense_1)
    dense_1 = Dense(nclass, activation=activations.sigmoid, name="dense_3_ptbdb")(dense_1)

    model = models.Model(inputs=inp, outputs=dense_1)
    opt = optimizers.Adam(0.001)

    model.compile(optimizer=opt, loss=losses.binary_crossentropy, metrics=['acc'])
    model.summary()
    return model

model = get_model()
file_path = "baseline_cnn_ptbdb.h5"
checkpoint = ModelCheckpoint(file_path, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
early = EarlyStopping(monitor="val_acc", mode="max", patience=5, verbose=1)
redonplat = ReduceLROnPlateau(monitor="val_acc", mode="max", patience=3, verbose=2)
callbacks_list = [checkpoint, early, redonplat]  # early

model.fit(X, Y, epochs=1000, verbose=2, callbacks=callbacks_list, validation_split=0.1)
model.save('ptbdb_baseline.h5')


Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 187, 1)]          0         
_________________________________________________________________
conv1d (Conv1D)              (None, 183, 16)           96        
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 179, 16)           1296      
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 89, 16)            0         
_________________________________________________________________
dropout (Dropout)            (None, 89, 16)            0         
_________________________________________________________________
conv1d_2 (Conv1D) 

In [0]:
df_train = pd.read_csv("mitbih_train.csv", header=None)
df_train = df_train.sample(frac=1)
df_test = pd.read_csv("mitbih_test.csv", header=None)

Y = np.array(df_train[187].values).astype(np.int8)
X = np.array(df_train[list(range(187))].values)[..., np.newaxis]

Y_test = np.array(df_test[187].values).astype(np.int8)
X_test = np.array(df_test[list(range(187))].values)[..., np.newaxis]

def get_model():
    nclass = 5
    inp = Input(shape=(187, 1))
    img_1 = Convolution1D(16, kernel_size=5, activation=activations.relu, padding="valid")(inp)
    img_1 = Convolution1D(16, kernel_size=5, activation=activations.relu, padding="valid")(img_1)
    img_1 = MaxPool1D(pool_size=2)(img_1)
    img_1 = Dropout(rate=0.1)(img_1)
    img_1 = Convolution1D(32, kernel_size=3, activation=activations.relu, padding="valid")(img_1)
    img_1 = Convolution1D(32, kernel_size=3, activation=activations.relu, padding="valid")(img_1)
    img_1 = MaxPool1D(pool_size=2)(img_1)
    img_1 = Dropout(rate=0.1)(img_1)
    img_1 = Convolution1D(32, kernel_size=3, activation=activations.relu, padding="valid")(img_1)
    img_1 = Convolution1D(32, kernel_size=3, activation=activations.relu, padding="valid")(img_1)
    img_1 = MaxPool1D(pool_size=2)(img_1)
    img_1 = Dropout(rate=0.1)(img_1)
    img_1 = Convolution1D(256, kernel_size=3, activation=activations.relu, padding="valid")(img_1)
    img_1 = Convolution1D(256, kernel_size=3, activation=activations.relu, padding="valid")(img_1)
    img_1 = GlobalMaxPool1D()(img_1)
    img_1 = Dropout(rate=0.2)(img_1)

    dense_1 = Dense(64, activation=activations.relu, name="dense_1")(img_1)
    dense_1 = Dense(64, activation=activations.relu, name="dense_2")(dense_1)
    dense_1 = Dense(nclass, activation=activations.softmax, name="dense_3_mitbih")(dense_1)

    model = models.Model(inputs=inp, outputs=dense_1)
    opt = optimizers.Adam(0.001)

    model.compile(optimizer=opt, loss=losses.sparse_categorical_crossentropy, metrics=['acc'])
    model.summary()
    return model

model = get_model()
file_path = "baseline_cnn_mitbih.h5"
checkpoint = ModelCheckpoint(file_path, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
early = EarlyStopping(monitor="val_acc", mode="max", patience=5, verbose=1)
redonplat = ReduceLROnPlateau(monitor="val_acc", mode="max", patience=3, verbose=2)
callbacks_list = [checkpoint, early, redonplat]  # early

model.fit(X, Y, epochs=1000, verbose=2, callbacks=callbacks_list, validation_split=0.1)
model.save('mitbih_baseline.h5')

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 187, 1)]          0         
_________________________________________________________________
conv1d_8 (Conv1D)            (None, 183, 16)           96        
_________________________________________________________________
conv1d_9 (Conv1D)            (None, 179, 16)           1296      
_________________________________________________________________
max_pooling1d_3 (MaxPooling1 (None, 89, 16)            0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 89, 16)            0         
_________________________________________________________________
conv1d_10 (Conv1D)           (None, 87, 32)            1568      
_________________________________________________________________
conv1d_11 (Conv1D)           (None, 85, 32)            3104

In [0]:
def make_differences(X):
    differences=np.zeros((len(X),len(X[1])))
    signals=np.zeros(len(X))
    for beat in range(len(X)):
        for signal in range(1,len(X[1])):
            if X[beat][signal-1]==0 and X[beat][signal]==0:
                if (signal>=len(X[1])-2):
                    signals[beat]=signal
                    break
                else:
                    if X[beat][signal+1]==0 and X[beat][signal+2]==0:
                        signals[beat]=signal
                        break
            differences[beat][signal]=X[beat][signal]-X[beat][signal-1]
    return differences,signals

window=15
def make_peaks(data):
    peaks=np.zeros((len(data),2*window))
    for i in range(len(data)):
        #I started after 10 because sometimes there is a big peak at the very start
        peak=np.argmax(data[i][10:])+10
        peakinfo=data[i][peak-window:peak+window]
        for j in range(len(peakinfo)):
            peaks[i][j]=peakinfo[j]
    return pd.DataFrame(peaks)

ptbdb_normal = pd.read_csv("ptbdb_normal.csv", header=None)
ptbdb_abnormal = pd.read_csv("ptbdb_abnormal.csv", header=None)
ptbdb = pd.concat([ptbdb_normal, ptbdb_abnormal])

mitbih_train = pd.read_csv("mitbih_train.csv", header=None)
mitbih_train = mitbih_train.sample(frac=1)
mitbih_test = pd.read_csv("mitbih_test.csv", header=None)


ptbdb_categories=2
ptbdb_train, ptbdb_test = train_test_split(ptbdb, test_size=0.2, random_state=42, stratify=ptbdb[187])
ptbdb_train_Y = np.array(ptbdb_train[187].values).astype(np.int8)
ptbdb_train_X = np.array(ptbdb_train[list(range(187))].values)[..., np.newaxis]
ptbdb_validation_x, ptbdb_train_x, ptbdb_validation_y, ptbdb_train_y = train_test_split(ptbdb_train_X, ptbdb_train_Y, test_size=0.33, random_state=42)
ptbdb_test_Y = np.array(ptbdb_test[187].values).astype(np.int8)
ptbdb_test_X = np.array(ptbdb_test[list(range(187))].values)[..., np.newaxis]
ptbdb_beats=range(len(ptbdb_train_x))
ptbdb_color=['green','red']
ptbdb_label=["Normal beat", "Abnormal beat"]

mitbih_categories=5
mitbih_train_Y = np.array(mitbih_train[187].values).astype(np.int8)
mitbih_train_X = np.array(mitbih_train[list(range(187))].values)[..., np.newaxis]
mitbih_validation_x, mitbih_train_x, mitbih_validation_y, mitbih_train_y = train_test_split(mitbih_train_X, mitbih_train_Y, test_size=0.33, random_state=42)  

ptbdb_peaks=make_peaks(ptbdb_train_X)
ptbdb_differences, ptbdb_signals=make_differences(ptbdb_train_X)

mitbih_differences, mitbih_signals=make_differences(mitbih_train_X)
mitbih_peaks=make_peaks(mitbih_train_X)

In [0]:
def ptbdb_scores(model,X,Y):
    pred_test = model.predict(X)
    test_probs = pred_test
    pred_test = np.argmax(pred_test, axis=-1)
    acc = accuracy_score(Y, pred_test)

    auroc = roc_auc_score(Y, test_probs)

    precision, recall, thresh = precision_recall_curve(Y,test_probs)
    auprc=auc(recall,precision)
    return acc,auroc,auprc    

class MyHyperModel(HyperModel):

    def __init__(self, input_size, num_classes):
        self.input_size = input_size
        self.num_classes = num_classes

    def build(self, hp):
        gate_type = hp.Choice('gate_type', ['SimpleRNN', 'GRU','LSTM', 'Bidirectional'])
        dropout_rate=rate=hp.Float(
            'dropout',
            min_value=0.0,
            max_value=0.5,
            default=0.25,
            step=0.1,
        )
        recurrent_dropout_rate=rate=hp.Float(
            'recurrent_dropout',
            min_value=0.0,
            max_value=0.5,
            default=0.25,
            step=0.1,
        )
        model = keras.Sequential()
        model.add(keras.layers.Masking(mask_value=0.,
                                  input_shape=(self.input_size[0], 1)))
        units=hp.Int('units', 12, self.input_size[0], 12)
        for i in range(hp.Int('num_layers', 0, 4)):
            if gate_type=='SimpleRNN':
                model.add(SimpleRNN(units,return_sequences=True,dropout=dropout_rate,recurrent_dropout=recurrent_dropout_rate))
            elif gate_type=='GRU':
                model.add(GRU(units,return_sequences=True,dropout=dropout_rate,recurrent_dropout=recurrent_dropout_rate))
            elif gate_type=='LSTM':
                model.add(LSTM(units,return_sequences=True,dropout=dropout_rate,recurrent_dropout=recurrent_dropout_rate))
            elif gate_type=='Bidirectional':
                model.add(Bidirectional(LSTM(units,return_sequences=True,dropout=dropout_rate,recurrent_dropout=recurrent_dropout_rate)))
        if gate_type=='SimpleRNN':
            model.add(SimpleRNN(units,dropout=dropout_rate,recurrent_dropout=recurrent_dropout_rate))
        elif gate_type=='GRU':
            model.add(GRU(units,dropout=dropout_rate,recurrent_dropout=recurrent_dropout_rate))
        elif gate_type=='LSTM':
            model.add(LSTM(units,dropout=dropout_rate,recurrent_dropout=recurrent_dropout_rate))
        elif gate_type=='Bidirectional':
            model.add(Bidirectional(LSTM(units,dropout=dropout_rate,recurrent_dropout=recurrent_dropout_rate)))
        model.add(Dense(64, activation=activations.relu, name="dense_1"))
        model.add(Dense(64, activation=activations.relu, name="dense_2"))
        model.add(Dense(self.num_classes-1, activation=activations.softmax, name="dense_3_mitbih"))
        optimizer_type=hp.Choice('optimizer',['SGD', 'RMSprop', 'ADAM'])
        learning_rate=hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4])
        if optimizer_type=='SGD':
            optimizer=keras.optimizers.SGD(learning_rate=learning_rate)
        elif optimizer_type=='ADAM':
            optimizer=keras.optimizers.Adam(
                learning_rate=learning_rate)
        elif optimizer_type=='RMSprop':
            optimizer=keras.optimizers.RMSprop(
                learning_rate=learning_rate)
        if self.num_classes >2:
            loss=losses.sparse_categorical_crossentropy
        else:
            loss=losses.binary_crossentropy
        model.compile(
            optimizer=optimizer,
            loss=loss,
            metrics=['acc'])
        return model

ptbdb_diff, trash=make_differences(ptbdb_train_X)
ptbdb_normal_tuner = Hyperband(
    MyHyperModel(input_size=ptbdb_train_X[0].shape, num_classes=2),
    objective='val_acc', seed=36,
    max_epochs=10,
    directory='ptbdb_normal')

ptbdb_peaks_tuner = Hyperband(
    MyHyperModel(input_size=ptbdb_peaks.T[0].shape, num_classes=2),
    objective='val_acc', seed=36,
    max_epochs=10,
    directory='ptbdb_peaks')

ptbdb_diff_tuner=Hyperband(
    MyHyperModel(input_size=ptbdb_diff[0].shape, num_classes=2),
    objective='val_acc', seed=36,
    max_epochs=10,
    directory='ptbdb_diff')

ptbdb_normal_tuner.search(ptbdb_train_X,
             y=ptbdb_train_Y,
             epochs=5,
             validation_split=0.2)
print('normal is done,yay.')
ptbdb_peaks_tuner.search(ptbdb_peaks.T,
             y=ptbdb_train_Y,
             epochs=5,
             validation_split=0.2)
ptbdb_diff_tuner.search(ptbdb_diff,
             y=ptbdb_train_Y,
             epochs=5,
             validation_split=0.2)
ptbdb_test_diff, trash=make_differences(ptbdb_test_X)
ptbdb_normal_acc,ptbdb_normal_auroc,ptbdb_normal_auprc=ptbdb_scores(ptbdb_normal_tuner.get_best_models(num_models=1)[0],ptbdb_test_X,ptbdb_test_Y)
ptbdb_peaks_acc,ptbdb_peaks_auroc,ptbdb_peaks_auprc=ptbdb_scores(ptbdb_peaks_tuner.get_best_models(num_models=1)[0],make_peaks(ptbdb_test_X).T,ptbdb_test_Y)
ptbdb_diff_acc,ptbdb_diff_auroc,ptbdb_diff_auprc=ptbdb_scores(ptbdb_diff_tuner.get_best_models(num_models=1)[0],ptbdb_test_diff,ptbdb_test_Y)


mitbih_diff, trash=make_differences(mitbih_test_X)               
mitbih_normal_tuner = Hyperband(
    MyHyperModel(input_size=mitbih_train_X[0].shape, num_classes=5),
    objective='val_acc', seed=36,
    max_epochs=40,
    directory='mitbih_normal')
mitbih_peaks_tuner = Hyperband(
    MyHyperModel(input_size=mitbih_peaks.T[0].shape, num_classes=5),
    objective='val_acc', seed=36,
    max_epochs=40,
    directory='mitbih_peaks')
mitbih_diff_tuner = Hyperband(
    MyHyperModel(input_size=mitbih_diff[0].shape, num_classes=5),
    objective='val_acc', seed=36,
    max_epochs=40,
    directory='mitbih_diff')

mitbih_normal_tuner.search(mitbih_train_X,
             y=mitbih_train_Y,
             epochs=5,
             validation_split=0.2)

mitbih_peaks_tuner.search(mitbih_peaks.T,
             y=mitbih_train_Y,
             epochs=5,
             validation_split=0.2)
mitbih_diff_tuner.search(mitbih_diff,
             y=mitbih_train_Y,
             epochs=5,
             validation_split=0.2)
mitbih_normal_model=mitbih_normal_tuner.get_best_models(num_models=1)[0]
mitbih_peaks_model=mitbih_peaks_tuner.get_best_models(num_models=1)[0]
mitbih_diff_model=mitbih_diff_tuner.get_best_models(num_models=1)[0]

pred_test = mitbih_normal_model.predict(mitbih_test_X)
pred_test = np.argmax(pred_test, axis=-1)
mitbih_normal_acc = accuracy_score(mitbih_test_Y, pred_test)

pred_test = mitbih_peaks_model.predict(make_peaks(mitbih_test_X).T)
pred_test = np.argmax(pred_test, axis=-1)
mitbih_peaks_acc = accuracy_score(mitbih_test_Y, pred_test)
mitbih_test_diff, trash =make_differences(mitbih_test_X)
pred_test = mitbih_diff_model.predict(mitbih_test_diff)
pred_test = np.argmax(pred_test, axis=-1)
mitbih_diff_acc = accuracy_score(mitbih_test_Y, pred_test)

INFO:tensorflow:Reloading Oracle from existing project ptbdb_normal/untitled_project/oracle.json
INFO:tensorflow:Reloading Tuner from ptbdb_normal/untitled_project/tuner0.json
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Train on 9312 samples, validate on 2329 samples
Epoch 3/5
9312/9312 [==============================] - 1028s 110ms/sample - loss: 4.2960 - acc: 0.7198 - val_loss: 4.1119 - val_acc: 0.7304
Epoch 4/5
9312/9312 [==============================] - 1007s 108ms/sample - loss: 4.2960 - acc: 0.7198 - val_loss: 4.1119 - val_acc: 0.7304
Epoch 5/5
9312/9312 [==============================] - 1025s 110ms/sample - loss: 4.2960 - acc: 0.7198 - val_loss: 4.1119 - val_acc: 0.7304


Train on 9312 samples, validate on 2329 samples
Epoch 3/5
9312/9312 [==============================] - 226s 24ms/sample - loss: 4.2960 - acc: 0.7198 - val_loss: 4.1119 - val_acc: 0.7304
Epoch 4/5
9312/9312 [==============================] - 225s 24ms/sample - loss: 4.2960 - acc: 0.7198 - val_loss: 4.1119 - val_acc: 0.7304
Epoch 5/5
9312/9312 [==============================] - 224s 24ms/sample - loss: 4.2960 - acc: 0.7198 - val_loss: 4.1119 - val_acc: 0.7304


Train on 9312 samples, validate on 2329 samples
Epoch 3/5
9312/9312 [==============================] - 783s 84ms/sample - loss: 4.2960 - acc: 0.7198 - val_loss: 4.1119 - val_acc: 0.7304
Epoch 4/5
9312/9312 [==============================] - 776s 83ms/sample - loss: 4.2960 - acc: 0.7198 - val_loss: 4.1119 - val_acc: 0.7304
Epoch 5/5
9312/9312 [==============================] - 762s 82ms/sample - loss: 4.2960 - acc: 0.7198 - val_loss: 4.1119 - val_acc: 0.7304


Train on 9312 samples, validate on 2329 samples
Epoch 3/5
9312/9312 [==============================] - 1542s 166ms/sample - loss: 4.2960 - acc: 0.7198 - val_loss: 4.1119 - val_acc: 0.7304
Epoch 4/5
9312/9312 [==============================] - 1520s 163ms/sample - loss: 4.2960 - acc: 0.7198 - val_loss: 4.1119 - val_acc: 0.7304
Epoch 5/5
9312/9312 [==============================] - 1565s 168ms/sample - loss: 4.2960 - acc: 0.7198 - val_loss: 4.1119 - val_acc: 0.7304


Train on 9312 samples, validate on 2329 samples
Epoch 3/5
9312/9312 [==============================] - 468s 50ms/sample - loss: 4.2960 - acc: 0.7198 - val_loss: 4.1119 - val_acc: 0.7304
Epoch 4/5
9312/9312 [==============================] - 458s 49ms/sample - loss: 4.2960 - acc: 0.7198 - val_loss: 4.1119 - val_acc: 0.7304
Epoch 5/5
9312/9312 [==============================] - 454s 49ms/sample - loss: 4.2960 - acc: 0.7198 - val_loss: 4.1119 - val_acc: 0.7304


Train on 9312 samples, validate on 2329 samples
Epoch 3/5
9312/9312 [==============================] - 339s 36ms/sample - loss: 4.2960 - acc: 0.7198 - val_loss: 4.1119 - val_acc: 0.7304
Epoch 4/5
9312/9312 [==============================] - 327s 35ms/sample - loss: 4.2960 - acc: 0.7198 - val_loss: 4.1119 - val_acc: 0.7304
Epoch 5/5
9312/9312 [==============================] - 319s 34ms/sample - loss: 4.2960 - acc: 0.7198 - val_loss: 4.1119 - val_acc: 0.7304


Train on 9312 samples, validate on 2329 samples
Epoch 3/5
9312/9312 [==============================] - 613s 66ms/sample - loss: 4.2960 - acc: 0.7198 - val_loss: 4.1119 - val_acc: 0.7304
Epoch 4/5
9312/9312 [==============================] - 610s 65ms/sample - loss: 4.2960 - acc: 0.7198 - val_loss: 4.1119 - val_acc: 0.7304
Epoch 5/5
9312/9312 [==============================] - 615s 66ms/sample - loss: 4.2960 - acc: 0.7198 - val_loss: 4.1119 - val_acc: 0.7304


Train on 9312 samples, validate on 2329 samples
Epoch 3/5
9312/9312 [==============================] - 436s 47ms/sample - loss: 4.2960 - acc: 0.7198 - val_loss: 4.1119 - val_acc: 0.7304
Epoch 4/5
9312/9312 [==============================] - 433s 46ms/sample - loss: 4.2960 - acc: 0.7198 - val_loss: 4.1119 - val_acc: 0.7304
Epoch 5/5
9312/9312 [==============================] - 438s 47ms/sample - loss: 4.2960 - acc: 0.7198 - val_loss: 4.1119 - val_acc: 0.7304


Train on 9312 samples, validate on 2329 samples
Epoch 6/14
9312/9312 [==============================] - 336s 36ms/sample - loss: 4.2960 - acc: 0.7198 - val_loss: 4.1119 - val_acc: 0.7304
Epoch 7/14
9312/9312 [==============================] - 336s 36ms/sample - loss: 4.2960 - acc: 0.7198 - val_loss: 4.1119 - val_acc: 0.7304
Epoch 8/14
9312/9312 [==============================] - 325s 35ms/sample - loss: 4.2960 - acc: 0.7198 - val_loss: 4.1119 - val_acc: 0.7304
Epoch 9/14
9312/9312 [==============================] - 330s 35ms/sample - loss: 4.2960 - acc: 0.7198 - val_loss: 4.1119 - val_acc: 0.7304
Epoch 10/14
9312/9312 [==============================] - 334s 36ms/sample - loss: 4.2960 - acc: 0.7198 - val_loss: 4.1119 - val_acc: 0.7304
Epoch 11/14
9312/9312 [==============================] - 324s 35ms/sample - loss: 4.2960 - acc: 0.7198 - val_loss: 4.1119 - val_acc: 0.7304
Epoch 12/14
9312/9312 [==============================] - 321s 34ms/sample - loss: 4.2960 - acc: 0.7198 - val_loss: 4

Train on 9312 samples, validate on 2329 samples
Epoch 6/14
9312/9312 [==============================] - 667s 72ms/sample - loss: 4.2960 - acc: 0.7198 - val_loss: 4.1119 - val_acc: 0.7304
Epoch 7/14
9312/9312 [==============================] - 652s 70ms/sample - loss: 4.2960 - acc: 0.7198 - val_loss: 4.1119 - val_acc: 0.7304
Epoch 8/14
9312/9312 [==============================] - 660s 71ms/sample - loss: 4.2960 - acc: 0.7198 - val_loss: 4.1119 - val_acc: 0.7304
Epoch 9/14
8896/9312 [===========================>..] - ETA: 27s - loss: 4.3021 - acc: 0.7194

In [5]:
class MyHyperModel(HyperModel):

    def __init__(self, input_size, num_classes):
        self.input_size = input_size
        self.num_classes = num_classes

    def build(self, hp):
        gate_type = hp.Choice('gate_type', ['SimpleRNN', 'GRU','LSTM', 'Bidirectional'])
        dropout_rate=rate=hp.Float(
            'dropout',
            min_value=0.0,
            max_value=0.5,
            default=0.25,
            step=0.1,
        )
        recurrent_dropout_rate=rate=hp.Float(
            'recurrent_dropout',
            min_value=0.0,
            max_value=0.5,
            default=0.25,
            step=0.1,
        )
        model = keras.Sequential()
        model.add(keras.layers.Masking(mask_value=0.,
                                  input_shape=(self.input_size[0], 1)))
        units=hp.Int('units', 12, self.input_size[0], 12)
        for i in range(hp.Int('num_layers', 0, 4)):
            if gate_type=='SimpleRNN':
                model.add(SimpleRNN(units,return_sequences=True,dropout=dropout_rate,recurrent_dropout=recurrent_dropout_rate))
            elif gate_type=='GRU':
                model.add(GRU(units,return_sequences=True,dropout=dropout_rate,recurrent_dropout=recurrent_dropout_rate))
            elif gate_type=='LSTM':
                model.add(LSTM(units,return_sequences=True,dropout=dropout_rate,recurrent_dropout=recurrent_dropout_rate))
            elif gate_type=='Bidirectional':
                model.add(Bidirectional(LSTM(units,return_sequences=True,dropout=dropout_rate,recurrent_dropout=recurrent_dropout_rate)))
        if gate_type=='SimpleRNN':
            model.add(SimpleRNN(units,dropout=dropout_rate,recurrent_dropout=recurrent_dropout_rate))
        elif gate_type=='GRU':
            model.add(GRU(units,dropout=dropout_rate,recurrent_dropout=recurrent_dropout_rate))
        elif gate_type=='LSTM':
            model.add(LSTM(units,dropout=dropout_rate,recurrent_dropout=recurrent_dropout_rate))
        elif gate_type=='Bidirectional':
            model.add(Bidirectional(LSTM(units,dropout=dropout_rate,recurrent_dropout=recurrent_dropout_rate)))
        model.add(Dense(64, activation=activations.relu, name="dense_1"))
        model.add(Dense(64, activation=activations.relu, name="dense_2"))
        model.add(Dense(self.num_classes-1, activation=activations.softmax, name="dense_3_mitbih"))
        optimizer_type=hp.Choice('optimizer',['SGD', 'RMSprop', 'ADAM'])
        learning_rate=hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4])
        if optimizer_type=='SGD':
            optimizer=keras.optimizers.SGD(learning_rate=learning_rate)
        elif optimizer_type=='ADAM':
            optimizer=keras.optimizers.Adam(
                learning_rate=learning_rate)
        elif optimizer_type=='RMSprop':
            optimizer=keras.optimizers.RMSprop(
                learning_rate=learning_rate)
        if self.num_classes >2:
            loss=losses.sparse_categorical_crossentropy
        else:
            loss=losses.binary_crossentropy
        model.compile(
            optimizer=optimizer,
            loss=loss,
            metrics=['acc'])
        return model

mitbih_diff, trash=make_differences(mitbih_train_X)               
mitbih_normal_tuner = Hyperband(
    MyHyperModel(input_size=mitbih_train_X[0].shape, num_classes=5),
    objective='val_acc', seed=36,
    max_epochs=40,
    directory='mitbih_normal')
mitbih_peaks_tuner = Hyperband(
    MyHyperModel(input_size=mitbih_peaks.T[0].shape, num_classes=5),
    objective='val_acc', seed=36,
    max_epochs=40,
    directory='mitbih_peaks')
mitbih_diff_tuner = Hyperband(
    MyHyperModel(input_size=mitbih_diff[0].shape, num_classes=5),
    objective='val_acc', seed=36,
    max_epochs=40,
    directory='mitbih_diff')

mitbih_normal_tuner.search(mitbih_train_X,
             y=mitbih_train_Y,
             epochs=5,
             validation_split=0.2)

mitbih_peaks_tuner.search(mitbih_peaks.T,
             y=mitbih_train_Y,
             epochs=5,
             validation_split=0.2)
mitbih_diff_tuner.search(mitbih_diff,
             y=mitbih_train_Y,
             epochs=5,
             validation_split=0.2)
mitbih_normal_model=mitbih_normal_tuner.get_best_models(num_models=1)[0]
mitbih_peaks_model=mitbih_peaks_tuner.get_best_models(num_models=1)[0]
mitbih_diff_model=mitbih_diff_tuner.get_best_models(num_models=1)[0]

pred_test = mitbih_normal_model.predict(mitbih_test_X)
pred_test = np.argmax(pred_test, axis=-1)
mitbih_normal_acc = accuracy_score(mitbih_test_Y, pred_test)

pred_test = mitbih_peaks_model.predict(make_peaks(mitbih_test_X).T)
pred_test = np.argmax(pred_test, axis=-1)
mitbih_peaks_acc = accuracy_score(mitbih_test_Y, pred_test)
mitbih_test_diff, trash =make_differences(mitbih_test_X)
pred_test = mitbih_diff_model.predict(mitbih_test_diff)
pred_test = np.argmax(pred_test, axis=-1)
mitbih_diff_acc = accuracy_score(mitbih_test_Y, pred_test)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Train on 70043 samples, validate on 17511 samples
Epoch 1/2
70043/70043 [==============================] - 2859s 41ms/sample - loss: nan - acc: 0.8260 - val_loss: nan - val_acc: 0.8332
Epoch 2/2
70043/70043 [==============================] - 2834s 40ms/sample - loss: nan - acc: 0.8264 - val_loss: nan - val_acc: 0.8332


/usr/local/lib/python3.6/dist-packages/kerastuner/engine/metrics_tracking.py:92: RuntimeWarning: All-NaN axis encountered
  return np.nanmin(values)


Train on 70043 samples, validate on 17511 samples
Epoch 1/2
70043/70043 [==============================] - 641s 9ms/sample - loss: nan - acc: 0.8262 - val_loss: nan - val_acc: 0.8332
Epoch 2/2
70043/70043 [==============================] - 631s 9ms/sample - loss: nan - acc: 0.8264 - val_loss: nan - val_acc: 0.8332


/usr/local/lib/python3.6/dist-packages/kerastuner/engine/metrics_tracking.py:92: RuntimeWarning: All-NaN axis encountered
  return np.nanmin(values)


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Train on 70043 samples, validate on 17511 samples
Epoch 1/2
70043/70043 [==============================] - 9073s 130ms/sample - loss: nan - acc: 0.8261 - val_loss: nan - val_acc: 0.8332
Epoch 2/2
70043/70043 [==============================] - 9145s 131ms/sample - loss: nan - acc: 0.8264 - val_loss: nan - val_acc: 0.8332


/usr/local/lib/python3.6/dist-packages/kerastuner/engine/metrics_tracking.py:92: RuntimeWarning: All-NaN axis encountered
  return np.nanmin(values)


Train on 70043 samples, validate on 17511 samples
Epoch 1/2
11456/70043 [===>..........................] - ETA: 12:04 - loss: nan - acc: 0.8245

KeyboardInterrupt: ignored

In [0]:
class MyHyperModel(HyperModel):

    def __init__(self, input_size, num_classes):
        self.input_size = input_size
        self.num_classes = num_classes

    def build(self, hp):
        gate_type = hp.Choice('gate_type', ['SimpleRNN', 'GRU','LSTM', 'Bidirectional'])
        dropout_rate=rate=hp.Float(
            'dropout',
            min_value=0.0,
            max_value=0.5,
            default=0.25,
            step=0.1,
        )
        recurrent_dropout_rate=rate=hp.Float(
            'recurrent_dropout',
            min_value=0.0,
            max_value=0.5,
            default=0.25,
            step=0.1,
        )
        model = keras.Sequential()
        model.add(keras.layers.Masking(mask_value=0.,
                                  input_shape=(self.input_size[0], 1)))
        units=hp.Int('units', 12, self.input_size[0], 12)
        for i in range(hp.Int('num_layers', 0, 4)):
            if gate_type=='SimpleRNN':
                model.add(SimpleRNN(units,return_sequences=True,dropout=dropout_rate,recurrent_dropout=recurrent_dropout_rate))
            elif gate_type=='GRU':
                model.add(GRU(units,return_sequences=True,dropout=dropout_rate,recurrent_dropout=recurrent_dropout_rate))
            elif gate_type=='LSTM':
                model.add(LSTM(units,return_sequences=True,dropout=dropout_rate,recurrent_dropout=recurrent_dropout_rate))
            elif gate_type=='Bidirectional':
                model.add(Bidirectional(LSTM(units,return_sequences=True,dropout=dropout_rate,recurrent_dropout=recurrent_dropout_rate)))
        if gate_type=='SimpleRNN':
            model.add(SimpleRNN(units,dropout=dropout_rate,recurrent_dropout=recurrent_dropout_rate))
        elif gate_type=='GRU':
            model.add(GRU(units,dropout=dropout_rate,recurrent_dropout=recurrent_dropout_rate))
        elif gate_type=='LSTM':
            model.add(LSTM(units,dropout=dropout_rate,recurrent_dropout=recurrent_dropout_rate))
        elif gate_type=='Bidirectional':
            model.add(Bidirectional(LSTM(units,dropout=dropout_rate,recurrent_dropout=recurrent_dropout_rate)))
        model.add(Dense(64, activation=activations.relu, name="dense_1"))
        model.add(Dense(64, activation=activations.relu, name="dense_2"))
        model.add(Dense(self.num_classes-1, activation=activations.softmax, name="dense_3_mitbih"))
        optimizer_type=hp.Choice('optimizer',['SGD', 'RMSprop', 'ADAM'])
        learning_rate=hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4])
        if optimizer_type=='SGD':
            optimizer=keras.optimizers.SGD(learning_rate=learning_rate)
        elif optimizer_type=='ADAM':
            optimizer=keras.optimizers.Adam(
                learning_rate=learning_rate)
        elif optimizer_type=='RMSprop':
            optimizer=keras.optimizers.RMSprop(
                learning_rate=learning_rate)
        if self.num_classes >2:
            loss=losses.sparse_categorical_crossentropy
        else:
            loss=losses.binary_crossentropy
        model.compile(
            optimizer=optimizer,
            loss=loss,
            metrics=['acc'])
        return model

mitbih_diff, trash=make_differences(mitbih_train_X)               
mitbih_normal_tuner = Hyperband(
    MyHyperModel(input_size=mitbih_train_X[0].shape, num_classes=5),
    objective='val_acc', seed=36,
    max_epochs=40,
    directory='mitbih_normal')
mitbih_peaks_tuner = Hyperband(
    MyHyperModel(input_size=mitbih_peaks.T[0].shape, num_classes=5),
    objective='val_acc', seed=36,
    max_epochs=40,
    directory='mitbih_peaks')
mitbih_diff_tuner = Hyperband(
    MyHyperModel(input_size=mitbih_diff[0].shape, num_classes=5),
    objective='val_acc', seed=36,
    max_epochs=40,
    directory='mitbih_diff')

#mitbih_normal_tuner.search(mitbih_train_X,
#             y=mitbih_train_Y,
#             epochs=5,
#             validation_split=0.2)

mitbih_peaks_tuner.search(mitbih_peaks.to_numpy().reshape(mitbih_peaks.shape[0],mitbih_peaks.shape[1],1),
             y=mitbih_train_Y,
             epochs=5,
             validation_split=0.2)
mitbih_diff_tuner.search(mitbih_diff,
             y=mitbih_train_Y,
             epochs=5,
             validation_split=0.2)
mitbih_normal_model=mitbih_normal_tuner.get_best_models(num_models=1)[0]
mitbih_peaks_model=mitbih_peaks_tuner.get_best_models(num_models=1)[0]
mitbih_diff_model=mitbih_diff_tuner.get_best_models(num_models=1)[0]

pred_test = mitbih_normal_model.predict(mitbih_test_X)
pred_test = np.argmax(pred_test, axis=-1)
mitbih_normal_acc = accuracy_score(mitbih_test_Y, pred_test)

pred_test = mitbih_peaks_model.predict(make_peaks(mitbih_test_X).T)
pred_test = np.argmax(pred_test, axis=-1)
mitbih_peaks_acc = accuracy_score(mitbih_test_Y, pred_test)
mitbih_test_diff, trash =make_differences(mitbih_test_X)
pred_test = mitbih_diff_model.predict(mitbih_test_diff)
pred_test = np.argmax(pred_test, axis=-1)
mitbih_diff_acc = accuracy_score(mitbih_test_Y, pred_test)

INFO:tensorflow:Reloading Oracle from existing project mitbih_normal/untitled_project/oracle.json
INFO:tensorflow:Reloading Tuner from mitbih_normal/untitled_project/tuner0.json
INFO:tensorflow:Reloading Oracle from existing project mitbih_peaks/untitled_project/oracle.json
Train on 70043 samples, validate on 17511 samples
Epoch 1/2
70043/70043 [==============================] - 429s 6ms/sample - loss: nan - acc: 0.8262 - val_loss: nan - val_acc: 0.8332
Epoch 2/2
70043/70043 [==============================] - 430s 6ms/sample - loss: nan - acc: 0.8264 - val_loss: nan - val_acc: 0.8332


/usr/local/lib/python3.6/dist-packages/kerastuner/engine/metrics_tracking.py:92: RuntimeWarning: All-NaN axis encountered
  return np.nanmin(values)


Train on 70043 samples, validate on 17511 samples
Epoch 1/2
70043/70043 [==============================] - 101s 1ms/sample - loss: nan - acc: 0.8261 - val_loss: nan - val_acc: 0.8332
Epoch 2/2
70043/70043 [==============================] - 97s 1ms/sample - loss: nan - acc: 0.8264 - val_loss: nan - val_acc: 0.8332


/usr/local/lib/python3.6/dist-packages/kerastuner/engine/metrics_tracking.py:92: RuntimeWarning: All-NaN axis encountered
  return np.nanmin(values)


Train on 70043 samples, validate on 17511 samples
Epoch 1/2
70043/70043 [==============================] - 1427s 20ms/sample - loss: nan - acc: 0.8263 - val_loss: nan - val_acc: 0.8332
Epoch 2/2
70043/70043 [==============================] - 1450s 21ms/sample - loss: nan - acc: 0.8264 - val_loss: nan - val_acc: 0.8332


/usr/local/lib/python3.6/dist-packages/kerastuner/engine/metrics_tracking.py:92: RuntimeWarning: All-NaN axis encountered
  return np.nanmin(values)


Train on 70043 samples, validate on 17511 samples
Epoch 1/2
70043/70043 [==============================] - 165s 2ms/sample - loss: nan - acc: 0.8261 - val_loss: nan - val_acc: 0.8332
Epoch 2/2
70043/70043 [==============================] - 166s 2ms/sample - loss: nan - acc: 0.8264 - val_loss: nan - val_acc: 0.8332


/usr/local/lib/python3.6/dist-packages/kerastuner/engine/metrics_tracking.py:92: RuntimeWarning: All-NaN axis encountered
  return np.nanmin(values)


Train on 70043 samples, validate on 17511 samples
Epoch 1/2
70043/70043 [==============================] - 1114s 16ms/sample - loss: nan - acc: 0.8262 - val_loss: nan - val_acc: 0.8332
Epoch 2/2
39744/70043 [================>.............] - ETA: 7:13 - loss: nan - acc: 0.8256Buffered data was truncated after reaching the output size limit.

In [10]:
mitbih_peaks.to_numpy().reshape(mitbih_peaks.shape[0],mitbih_peaks.shape[1],1).shape

(87554, 30, 1)